In [1]:
# CO 370 Group Project Code
# This codes reads in a CSV file, then performs the calculations

# Sources used:
# https://www.w3schools.com/python/pandas/pandas_csv.asp
# https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe 
# https://stackoverflow.com/questions/1541797/how-do-i-check-if-there-are-duplicates-in-a-flat-list
# https://docs.gurobi.com/projects/optimizer/en/current/reference/python/linexpr.html

In [2]:
# Imports
from gurobipy import *
import pandas as pd # Used to read csv files. 

In [3]:
# Variables to set
PRODUCT_CSV = "product_parameter.csv" # This is the file name of the csv that contains all of the products along with their associated parameters
SUPPLEMENT_PRODUCT_CSV = "env_product_supplement.csv" # TODO: Look at combining this with the above
SHELF_CSV = "env_shelves-Copy1.csv" # TODO: add description
SCALAR_CSV = "env_scalars.csv"
NUMBER_LEVELS_PER_SHELF = 5 # This is the number of levels on each shelving
NUMBER_SLOTS_PER_SHELF = 50 # This is the number of product slots each shelf can hold
MAX_SUPPLIERS_PER_PRODUCT = 3
LARGE_NUMBER = 1000000

In [4]:
# Variables set by script:
NUMBER_OF_SHELVES = 35 # Redefine in CSV TODO remove default
PREDICT_CUSTOMERS_PER_DAY = 500
IMPULSE_SCALAR_MULT_FOR_HIGH_DEMAND_SHELF = 1
SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I = []

In [5]:
# Product class, used to define the parameters for each product
class product:
    def __init__(self):
        self.product_name = "" # This stores the name of each product
        self.product_supplier = "" # This is the supplier of the product (NOTE: it is possible for us to have several products with same name from different suppliers)
        self.shelf_type = "" # This is the shelf type, for example R, S or F
        self.is_essential_item = False # This determines if this product makes up high traffic route
        self.product_category = "" # This is a string with the product category, for example "fish"
        self.profit_per_unit = -1 # This is the profit made by selling 1 unit of the product
        self.prob_of_intentional_purchase = -1 # This is in [0,1] and indicates the probability that a customer intends to purchase this product
                                          # This is for example the probability that someone has the item on their shopping list and hence will buy it
                                          # regardless of how it is displayed
        self.prob_of_impulse_purchase = -1 # This is in [0,1] and indicates the proabability that a customer would impulsively purchase a product
                                      # TODO add more details, but this must be scaled significantly to be meaningful
        self.slots_per_display = -1 # This represents the amount of space each display of a product takes up
        self.min_display = -1 # This represents the minimum number of displays that can be put up of a product
                         # This will likely be 0, but accomodates the case that a brand refuses to have a small display in the store
        self.max_display = -1 # This represents the maximum number of displays that can be put up of a product
                         # This for example would represent the case where stock is limited
        self.units_per_display = -1 # This represents how many units of a product are in its display
        self.price_to_display_on_shelf_i_not_on_high_traffic = [] # price_to_display_on_shelf_i_not_on_high_traffic[i - 1] represens how much a brand is willing to pay to have their product placed on shelf level i while not on high traffic route
        self.price_to_display_on_shelf_i_on_high_traffic = [] # price_to_display_on_shelf_i_on_high_traffic[i - 1] represens how much a brand is willing to pay to have their product placed on shelf level i while on a high traffic route

    def get_product_name(self):
        return self.product_name
    
    # Guirobi discourages the use of spaces in variables names, so this removes them, see: https://docs.gurobi.com/projects/optimizer/en/13.0/reference/python/model.html
    def get_safe_product_name(self):
        return self.product_name.replace(" ", "_")

    def get_product_supplier(self):
        return self.product_supplier

    def get_product_id(self):
        product_id = f"{self.product_name} {self.product_supplier}"
        return product_id.replace(" ", "_")

    def get_safe_category(self):
        return self.product_category.replace(" ", "_")

    def get_safe_shelf_type(self):
        return self.shelf_type.replace(" ", "_")

    def get_is_essential_item(self):
        return self.is_essential_item

    def get_slots_per_display(self):
        return self.slots_per_display
    
    def get_max_display(self):
        return self.max_display

    def get_min_display(self):
        return self.min_display

    def get_units_per_display(self):
        return self.units_per_display

    def get_profit_per_unit(self):
        return self.profit_per_unit
        
    def get_prob_of_intentional_purchase(self):
        return self.prob_of_intentional_purchase

    def get_prob_of_impulse_purchase(self):
        return self.prob_of_impulse_purchase

    def get_price_to_display_on_shelf_level_i_high_traffic(self, i):
        return self.price_to_display_on_shelf_i_on_high_traffic[i - 1] / 8

    def get_price_to_display_on_shelf_level_i_not_on_high_traffic(self, i):
        return self.price_to_display_on_shelf_i_not_on_high_traffic[i - 1] / 8
        
    def print_product(self):
        print(f"product_id: {self.get_product_id()}")
        print(f"product_name: {self.product_name}")
        print(f"product_supplier: {self.product_supplier}")
        print(f"product_category: {self.product_category}")
        print(f"shelf_type: {self.shelf_type}")
        print(f"is_essential_item: {self.is_essential_item}")
        print(f"profit_per_unit: {self.profit_per_unit}")
        print(f"prob_of_intentional_purchase: {self.prob_of_intentional_purchase}")
        print(f"prob_of_impulse_purchase: {self.prob_of_impulse_purchase}")
        print(f"slots_per_display: {self.slots_per_display}")
        print(f"min_display: {self.min_display}")
        print(f"max_display: {self.max_display}")
        print(f"units_per_display: {self.units_per_display}")
        for i in range(len(self.price_to_display_on_shelf_i_not_on_high_traffic)):
            print(f"price_to_display_on_shelf_i_not_on_high_traffic[{i}]: {self.price_to_display_on_shelf_i_not_on_high_traffic[i]}")
        for i in range(len(self.price_to_display_on_shelf_i_on_high_traffic)):
            print(f"price_to_display_on_shelf_i_on_high_traffic[{i}]: {self.price_to_display_on_shelf_i_on_high_traffic[i]}")
        
    def set_product_name(self, new_product_name):
        retVal = True
        new_product_name_str = ""
        try:
            new_product_name_str = str(new_product_name)
        except:
            print(f"An error occurred while calling set_product_name, unable to convert passed to str")
            retVal = False
            return retVal
        if new_product_name_str == "":
            print("Potential error occured when set_product_name was called, given string was empty")
            retVal = False
        self.product_name = new_product_name_str
        
        return retVal

    def set_product_supplier(self, supplier):
        retVal = True
        supplier_str = ""
        try:
            supplier_str = str(supplier)
        except:
            print(f"An error occurred while calling set_product_supplier, unable to convert passed shelf_type to str")
            retVal = False
            return retVal
        self.product_supplier = supplier_str
        return retVal
        
    def set_shelf_type(self, shelf_type):
        retVal = True
        shelf_type_str = ""
        try:
            shelf_type_str = str(shelf_type)
        except:
            print(f"An error occurred while calling set_shelf_type, unavle to convert passed shelf_type to str")
            retVal = False
            return retVal
        if shelf_type_str == "":
            print("Error occured when set_shelf_type was called, given string was empty")
            retVal = False
        self.shelf_type = shelf_type_str
        return retVal

    def set_is_essential_item(self, is_essential):
        retVal = True
        is_essential_bool = False
        try:
            is_essential_bool = bool(is_essential)
        except:
            print(f"An error occurred while calling set_is_essential_item, unavle to convert passed is_essential to bool")
            retVal = False
            return retVal
        self.is_essential_item = is_essential_bool
        return retVal
        
    
    def set_product_category(self, product_category):
        retVal = True
        product_category_str = ""
        try:
            product_category_str = str(product_category)
        except:
            print(f"An error occurred while calling set_product_category, unable to convert passed to str")
            retVal = False
            return retVal
        if product_category_str == "":
            print("Error occured when set_product_category was called, given string was empty")
            retVal = False
        self.product_category = product_category_str
        
        return retVal
        

    def set_profit_per_unit(self, profit_per_unit):
        retVal  = True
        float_set_profit_per_unit = -1
        try:
            float_set_profit_per_unit = float(profit_per_unit)
        except:
            print(f"An error occurred while calling set_profit_per_unit for {self.product_name}, unable to convert profit_per_unit to float, recieved: {profit_per_unit}")
            retVal = False
            return retVal
        if float_set_profit_per_unit < 0: # Note: technically we could allow this for the case where the manufacturer pays lots for a display
            print(f"An error occurred, profit for product {self.product_name} was less than 0, this is not allowed, as in this case the product should not be stocked")
            retVal = False
        self.profit_per_unit = float_set_profit_per_unit
        return retVal

    def set_prob_of_intentional_purchase(self, prob_of_intentional_purchase):
        retVal = True
        float_prob_of_intentional_purchase = -1
        try:
            float_prob_of_intentional_purchase = float(prob_of_intentional_purchase)
        except:
            print(f"An error occurred while calling set_prob_of_intentional_purchase for {self.product_name}, unable to convert prob_of_intentional_purchase to float, recieved: {prob_of_intentional_purchase}")
            retVal = False
            return retVal
        if float_prob_of_intentional_purchase < 0 or float_prob_of_intentional_purchase > 1:
            print(f"An error occurred, prob of intentional purchase for product {self.product_name} was not in [0,1]")
            retVal = False
        self.prob_of_intentional_purchase = float_prob_of_intentional_purchase
        return retVal
        
    def set_prob_of_impulse_purchase(self, prob_of_impulse_purchase):
        retVal = True
        float_prob_of_impulse_purchase = -1
        try:
            float_prob_of_impulse_purchase = float(prob_of_impulse_purchase)
        except:
            print(f"An error occurred while calling set_prob_of_impulse_purchase for {self.product_name}, unable to convert prob_of_impulse_purchase to float, recieved: {prob_of_impulse_purchase}")
            retVal = False
            return retVal
        if float_prob_of_impulse_purchase < 0 or float_prob_of_impulse_purchase > 1:
            print(f"An error occurred, prob of impulse purchase for product {self.product_name} was not in [0,1]")
            retVal = False
        self.prob_of_impulse_purchase = float_prob_of_impulse_purchase
        return retVal
    
    def set_slots_per_display(self, slots_per_display):
        retVal = True
        int_slots_per_display = -1
        try:
            int_slots_per_display = int(slots_per_display)
        except:
            print(f"An error occurred while calling set_slots_per_display for {self.product_name}, unable to convert slots_per_display to an integer, recieved: {slots_per_display}")
            retVal = False
            return retVal
        if int_slots_per_display <= 0:
            printf(f"An error occured, value of slots_per_display for {self.product_name} is non-positive, this is not allowed each products display must take up atleast 1 space")
            retVal = False
        self.slots_per_display = int_slots_per_display
        return retVal
        
    def set_min_display(self, min_display):
        retVal = True
        int_min_display = -1
        try:
            int_min_display = int(min_display)
        except:
            print(f"An error occurred while calling set_min_display for {self.product_name}, unable to convert min_display to an integer, recieved: {min_display}")
            retVal = False
            return retVal
        if int_min_display < 0:
            printf(f"An error occured, value of min_display for {self.product_name} is negative")
            retVal = False
        self.min_display = int_min_display
        return retVal

    def set_max_display(self, max_display):
        retVal = True
        int_max_display = -1
        try:
            int_max_display = int(max_display)
        except:
            print(f"An error occurred while calling set_max_display for {self.product_name}, unable to convert max_display to an integer, recieved: {max_display}")
            retVal = False
            return retVal
        if int_max_display < 0: # Allowing 0 is allowed here, could consider banning it, if 0 we would not be stocking the product
            printf(f"An error occured, value of min_display for {self.product_name} is negative")
            retVal = False
        self.max_display = int_max_display
        return retVal

    def set_units_per_display(self, units_per_display):
        retVal = True
        int_units_per_display = -1
        try:
            int_units_per_display = int(units_per_display)
        except:
            print(f"An error occurred while calling set_units_per_display for {self.product_name}, unable to convert units_per_display to an integer, recieved: {units_per_display}")
            retVal = False
            return retVal
        if int_units_per_display <= 0: 
            printf(f"An error occured, value of units_per_display for {self.product_name} is non-positive") # A display must have atleast one product in it
            retVal = False
        self.units_per_display = int_units_per_display
        return retVal

    def set_price_to_display_on_shelf_i_not_on_high_traffic(self, arr):
        retVal = True
        if not isinstance(arr, list):
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, the parameter passed was not a list")
            retVal = False
            return retVal
        if len(arr) != NUMBER_LEVELS_PER_SHELF:
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, arr does not contain exactly {NUMBER_LEVELS_PER_SHELF} units, it contains {len(arr)}")
            retVal = False
            return retVal
        for display_price in arr:
            float_display_price = -1
            try:
                float_display_price = float(display_price)
            except:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, unable to covert an element to a float, recieved {display_price}")
            if float_display_price < 0:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_not_on_high_traffic, arr does contains a negative element")
                retVal = False
                return retVal 
            self.price_to_display_on_shelf_i_not_on_high_traffic.append(float_display_price)
        return retVal

    def set_price_to_display_on_shelf_i_on_high_traffic(self, arr):
        retVal = True
        if not isinstance(arr, list):
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, the parameter passed was not a list")
            retVal = False
            return retVal
        if len(arr) != NUMBER_LEVELS_PER_SHELF:
            print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, arr does not contain exactly {NUMBER_LEVELS_PER_SHELF} units, it contains {len(arr)}")
            retVal = False
            return retVal
        for display_price in arr:
            float_display_price = -1
            try:
                float_display_price = float(display_price)
            except:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, unable to covert an element to a float, recieved {display_price}")
            if float_display_price < 0:
                print(f"Error occurred while calling set_price_to_display_on_shelf_i_on_high_traffic, arr does contains a negative element")
                retVal = False
                return retVal
            self.price_to_display_on_shelf_i_on_high_traffic.append(float_display_price)
        return retVal

    # This is called to verify the product is properly initialized
    def isInitialized(self):
        retVal = True
        if not isinstance(self.product_name, str):
            print("product_name is not initialized to be a string")
            retVal = False
        elif self.product_name == "":
            print("product_name is the empty string string")
            retVal = False
        if not isinstance(self.product_supplier, str):
            print("product_supplier is not initialized to be a string")
            retVal = False
        elif self.product_supplier == "":
            print("product_supplier is the empty string string")
            retVal = False
        if not isinstance(self.profit_per_unit, float):
            print("profit_per_unit is not initialized to be a float")
            retVal = False
        elif self.profit_per_unit < 0:
            print("profit_per_unit is negative")
            retVal = False
        if not isinstance(self.prob_of_intentional_purchase, float):
            print("prob_of_intentional_purchase is not initialized to be a float")
            retVal = False
        elif self.prob_of_intentional_purchase < 0 or self.prob_of_intentional_purchase > 1:
            print("prob_of_intentional_purchase is not in [0,1]")
            retVal = False
        if not isinstance(self.slots_per_display, int):
            print("slots_per_display is not initialized to be a int")
            retVal = False
        if self.slots_per_display < 1:
            print("slots_per_display per display is non-positive")
            retVal = False
        if not isinstance(self.min_display, int):
            print("min_display is not initialized to be a int")
            retVal = False
        elif self.min_display < 0:
            print("min_display is negative")
            retVal = False
        if not isinstance(self.max_display, int):
            print("max_display is not initialized to be a int")
            retVal = False
        elif self.max_display < 1:
            print("max_display is non-positive")
            retVal = False
        if isinstance(self.min_display, int) and isinstance(self.max_display, int) and self.min_display > self.max_display:
            print("min_display is greater than max_display")
            retVal = False
        if not isinstance(self.units_per_display, int):
            print("units_per_display is not initialized to be a int")
            retVal = False
        elif self.units_per_display < 1:
            print("units_per_display is non-positive")
            retVal = False
        if not isinstance(self.price_to_display_on_shelf_i_not_on_high_traffic, list):
            print("price_to_display_on_shelf_i_not_on_high_traffic is not an instance of list")
            retVal = False
        elif len(self.price_to_display_on_shelf_i_not_on_high_traffic) != NUMBER_LEVELS_PER_SHELF:
            print("price_to_display_on_shelf_i_not_on_high_traffic does not have length NUMBER_LEVELS_PER_SHELF")
            retVal = False
        else:
            for display_price in self.price_to_display_on_shelf_i_not_on_high_traffic:
                if not isinstance(display_price, float):
                    print("A value of price_to_display_on_shelf_i_not_on_high_traffic is not a float")
                    retVal = False
                elif display_price < 0:
                    print("A value of price_to_display_on_shelf_i_not_on_high_traffic is negative")
                    retVal = False
        if not isinstance(self.price_to_display_on_shelf_i_on_high_traffic, list):
            print("price_to_display_on_shelf_i_on_high_traffic is not an instance of list")
            retVal = False
        elif len(self.price_to_display_on_shelf_i_on_high_traffic) != NUMBER_LEVELS_PER_SHELF:
            print("price_to_display_on_shelf_i_on_high_traffic does not have length NUMBER_LEVELS_PER_SHELF")
            retVal = False
        else:
            for display_price in self.price_to_display_on_shelf_i_on_high_traffic:
                if not isinstance(display_price, float):
                    print("A value of price_to_display_on_shelf_i_on_high_traffic is not a float")
                    retVal = False
                elif display_price < 0:
                    print("A value of price_to_display_on_shelf_i_on_high_traffic is negative")
                    retVal = False
        if not isinstance(self.is_essential_item, bool):
            print("is_essential_item is not initialized to be a bool")
            retVal = False 
        if not isinstance(self.shelf_type, str):
            print("shelf_type is not an instance of str")
            retVal = False
        elif self.shelf_type == "":
            print("shelf_type is the empty string")
            retVal = False
        return retVal

In [6]:
# This defines a shelf
# TODO: Clean it up to look like product
class shelf:
    def __init__(self, shelf_id, shelf_type, high_traffic):
        retval = True
        shelf_id_str = ""
        try:
            shelf_id_str = str(shelf_id)
        except:
            print("Error, unable to convert shelf_id to str")
            retval = False
            return retval
        if shelf_id_str == "":
            print("Error, shelf_id was the empty string")
            retval = False
            return retval
        
        self.shelf_id = shelf_id_str # This contains the ID of the shelf (could be a number or a letter)
        
        shelf_type_str = ""
        try:
            shelf_type_str = str(shelf_type)
        except:
            print("Error, unable to convert shelf_type to str")
            retval = False
            return retval
        if shelf_type_str == "":
            print("Error, shelf_type was the empty string")
            retval = False
            return retval
        self.shelf_type = shelf_type_str # This must be identical to the shelf type in the product class TODO fix
        
        high_traffic_bool = False
        try:
            high_traffic_bool = bool(high_traffic)
        except:
            print("Error, unable to convert high_traffic to bool")
        
        self.high_traffic = high_traffic # Determine if a shelf is a high traffic one

    def get_shelf_id(self):
        return self.shelf_id
    
    def get_shelf_type(self):
        return self.shelf_type.replace(' ', '_')

    def get_high_traffic(self):
        return self.high_traffic

    def print(self):
        print(f"shelf_id: {self.shelf_id}")
        print(f"shelf_type: {self.shelf_type}")
        print(f"high_traffic: {self.high_traffic}")

In [7]:
# Set by python script
products = [] # This is made up of product class items, one for every product
shelves = [] # This is made up of the set of shelves

In [8]:
# This script create a product class for every item found in the source_csv
# Requires the CSV file to be in the following format:
# Product Name, Supplier, Category, profit per unit, purchase probability, impulse probability, Space per display, items per display, minimum number of displays, maximum number of displays, price willing to pay for shelf placement on level i not on high traffic, price willing to pay for shelf placement on level i on high traffic (For all i in 1 to NUMBER_LEVELS_PER_SHELF)
# Note, the first line of file is not read, as this is likely to be the key for the table
def initialize_products(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows): 
        success = True
        product_object = product()
        success = product_object.set_product_name(csv_data.iloc[i,0]) and success
        success = product_object.set_product_supplier(csv_data.iloc[i,1]) and success
        success = product_object.set_product_category(csv_data.iloc[i,2]) and success
        success = product_object.set_profit_per_unit(csv_data.iloc[i,3]) and success
        success = product_object.set_prob_of_intentional_purchase(csv_data.iloc[i,4]) and success
        success = product_object.set_prob_of_impulse_purchase(csv_data.iloc[i,5]) and success
        success = product_object.set_slots_per_display(csv_data.iloc[i,6]) and success
        success = product_object.set_units_per_display(csv_data.iloc[i,7]) and success
        success = product_object.set_min_display(csv_data.iloc[i,8]) and success
        success = product_object.set_max_display(csv_data.iloc[i,9]) and success

        price_arr_for_none_high_traffic = []
        price_arr_for_none_high_traffic.clear()
        price_arr_for_high_traffic = []
        price_arr_for_high_traffic.clear()
        for k in range(2 * NUMBER_LEVELS_PER_SHELF):
            if k % 2 == 0: # ie k is even
                price_arr_for_none_high_traffic.append(csv_data.iloc[i,10+k])
            else:
                price_arr_for_high_traffic.append(csv_data.iloc[i,10+k])
        success = product_object.set_price_to_display_on_shelf_i_not_on_high_traffic(price_arr_for_none_high_traffic) and success
        success = product_object.set_price_to_display_on_shelf_i_on_high_traffic(price_arr_for_high_traffic) and success

        if not success:
            print("An error occurred whilst initializating the product, it is printed below")
            product_object.print_product()
            return
        products.append(product_object) 

def verify_product_initialisation():
    for product in products:
        if not product.isInitialized():
            print("Error, the following product is not initialised correctly")
            product.print_product()
            

# This is for for debugging, w
def print_all_products():
    for product in products:
        product.print_product()

# Given that the ids of products are used in variable names, it is essential all products have distinct ids, this checks that
# Heavily inspired by https://stackoverflow.com/questions/1541797/how-do-i-check-if-there-are-duplicates-in-a-flat-list
def check_for_product_id_duplicates():
    seen = set()
    for product in products:
        product_id = product.get_product_id()
        if product_id in seen:
            print(f"Error, product name {product_id} is used multiple times")
            return 
        seen.add(product_id)    

def check_for_shelf_id_duplicates():
    seen = set()
    for shelf in shelves:
        shelf_id = shelf.get_shelf_id()
        if shelf_id in seen:
            print(f"Error, product name {shelf_id} is used multiple times")
            return 
        seen.add(shelf_id)  

# Finds a product in product array, or prints an error
def find_in_product_array(product_name):
    product_matches = []
    for product in products:
        if product_name == product.get_product_name():
            product_matches.append(product)
    if not product_matches:
        print(f"ERROR: unable to find {product_name}")
    return product_matches

# Processes supplemental data, that being if a product is an essential item and what shelf type a product must be assigned to
# TODO detail format
def process_supplemental_product_info(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows):
        product_name = csv_data.iloc[i,0]
        shelf_type = csv_data.iloc[i,1]
        is_essential = False
        if csv_data.iloc[i,2] == 1:
            is_essential = True
        product_matches = find_in_product_array(product_name)
        for product in product_matches:
            product.set_product_name(product_name)
            product.set_shelf_type(shelf_type)
            product.set_is_essential_item(is_essential)

# Process shelf data CSV
# TODO explain format
def proccess_shelf_info(source_csv):
    csv_data = pd.read_csv(source_csv)
    number_rows = csv_data.shape[0]
    for i in range(0, number_rows):
        shelf_id = csv_data.iloc[i,0]
        shelf_type = csv_data.iloc[i,1]
        shelf_high_traffic = False
        if csv_data.iloc[i,2] == 1:
            shelf_high_traffic = True
        shelves.append(shelf(shelf_id, shelf_type, shelf_high_traffic))

# Proccess scalar data CSV
# TODO explain format
def process_scalar_info(source_csv):
    csv_data = pd.read_csv(source_csv)
    global PREDICT_CUSTOMERS_PER_DAY
    global IMPULSE_SCALAR_MULT_FOR_HIGH_DEMAND_SHELF
    global SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I
    PREDICT_CUSTOMERS_PER_DAY = csv_data.iloc[0,1]
    print( csv_data.iloc[0,1])
    
    IMPULSE_SCALAR_MULT_FOR_HIGH_DEMAND_SHELF = csv_data.iloc[1,1]
    SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I.clear() # Ensure it is empty
    SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I.append(0) # As we want it such that SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I[i] is the value for shelf i and no shelf 0 exists
    for i in range(NUMBER_LEVELS_PER_SHELF):
        SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I.append(csv_data.iloc[1+i,1])


In [9]:
# Initialise products array
initialize_products(PRODUCT_CSV)
process_supplemental_product_info(SUPPLEMENT_PRODUCT_CSV)
verify_product_initialisation()
check_for_product_id_duplicates()

# Initialise shelves array
proccess_shelf_info(SHELF_CSV)
check_for_shelf_id_duplicates()

# print_all_products() #Only want this if you are debugging
shelves_set = [] # Contains all IDs of the shelves
for shelf in shelves:
    shelves_set.append(shelf.get_shelf_id())
NUMBER_OF_SHELVES = len(shelves_set)

# for shelf in shelves:
#     shelf.print()

shelf_levels = list(range(1, NUMBER_LEVELS_PER_SHELF + 1)) # This simply contains all of the levels of each shelf
process_scalar_info(SCALAR_CSV)

5000.0


In [10]:
print(PREDICT_CUSTOMERS_PER_DAY)

5000.0


In [11]:
# print_all_products()

In [12]:
# Initialize the model
model = Model("RetailSpace")

Set parameter Username
Set parameter LicenseID to value 2709513
Academic license - for non-commercial use only - expires 2026-09-16


In [13]:
# Define variables that determine if product p is place on shelf b on level j
# is_product_i_on_shelf_j_level_k = model.addVars(product_id_list, range(1, NUMBER_OF_SHELVES), range(1, NUMBER_LEVELS_PER_SHELF), vtype=GRB.BINARY, name="is_product_i_on_shelf_j_level_k")

# Ensure that each product is assigned to atmost one shelf 
# (3.5.1.2)
# model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') <= 1 for product in product_id_list), name = "Ensure each product is on atmost on slot")

# Define variables that determine what shelf level a product is on (variables is 1 if product is on that shelf level, 0 otherwise)
# is_product_i_on_level_j = model.addVars(product_id_list, range(1, NUMBER_LEVELS_PER_SHELF), vtype=GRB.BINARY, name = "is_product_i_on_level_j")
# Define a constraint forcing above definition to be true, this is simply summing over all shelves of is_product_i_on_shelf_j_level_k for a product
# (3.5.1.5)
# model.addConstrs((is_product_i_on_level_j[product,level] == is_product_i_on_shelf_j_level_k.sum(product, '*', level) for product in product_id_list for level in range(1, NUMBER_LEVELS_PER_SHELF)), name = "Calculate shelf level")


In [14]:
# List of product ids
product_id_list = []
for product in products:
    product_id_list.append(product.get_product_id())

# Dictionary of product names (each key is a name, and each value is all products with that name, ie. the different suppliers)
product_name_dict = {}
for product in products:
    product_name = product.get_safe_product_name()
    if product_name not in product_name_dict:
        product_name_dict[product_name] = []    
    product_name_dict[product_name].append(product)

def print_product_name_dict():
    for product_name in product_name_dict:
        print(f"Key: {product_name}")
        print("Values:")
        for product in product_name_dict[product_name]:
            print(f"    - {product.get_product_id()}")

# List of Categories
categories = []
for product in products:
    if product.get_safe_category() not in categories:
        categories.append(product.get_safe_category())

# List of shelf types
shelf_types = []
for product in products:
    if product.get_safe_shelf_type() not in shelf_types:
        shelf_types.append(product.get_safe_shelf_type())

# List of essential items
essential_items = []
for product in products:
    if product.get_is_essential_item():
        if product.get_safe_product_name() not in essential_items:
            essential_items.append(product.get_safe_product_name())

In [15]:
# Variables

# This is x in the proposal
# This is set equal to 1 if product i is on shelf j level k and 0 otherwise
is_product_i_on_shelf_j_level_k = model.addVars(product_id_list, shelves_set, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_j_level_k")

# This is l in the proposal
# This is set to 1 if product i is on shelf level j and 0 otherwise
is_product_i_on_shelf_level_j = model.addVars(product_id_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_shelf_level_j")

# This is l' in the proposal (slightly modified)
# This is set to 1 if product i is on shevel level j and that shelf is a not high traffic one
is_product_i_on_low_traffic_shelf_level_j = model.addVars(product_id_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_low_traffic_shelf_level_j")

# This is l'' in the proposal (slightly modified)
# This is set to 1 if product i is on shevel level j and that shelf is a high traffic one
is_product_i_on_high_traffic_shelf_level_j = model.addVars(product_id_list, shelf_levels, vtype=GRB.BINARY, name="is_product_i_on_high_traffic_shelf_level_j")

# TODO DELETE: I am removing m as I am assuming they'll be charged, as there'd be no reason not to

# This is y in the proposal
# This is an non-negative integer denoting how many displays are assigned to product i
displays_assigned_to_product_i = model.addVars(product_id_list, vtype=GRB.INTEGER, lb=0, name="displays_assigned_to_product_i")

# This is z in the proposal
# This is 1 if product i is stocked by the store
is_product_i_stocked = model.addVars(product_id_list, vtype=GRB.BINARY, name="is_product_i_stocked")

# This is e in the proposal
# This is defined to be number of displays of product i on shelf j level k
displays_of_product_i_on_shelf_j_level_k = model.addVars(product_id_list, shelves_set, shelf_levels, vtype=GRB.INTEGER, lb=0, name="displays_of_product_i_on_shelf_j_level_k")

# TODO DELETE: I am removing f as it looks like g does its job

# This is g in the proposal
# This is the number of units of product on shelf i level j
total_units_on_shelf_i_level_j = model.addVars(shelves_set, shelf_levels, vtype=GRB.INTEGER, lb=0, name="total_units_on_shelf_i_level_j")

# This is h in the proposal
# This is 1 if category i is on shelf j
is_category_i_on_shelf_j = model.addVars(categories, shelves_set, vtype=GRB.BINARY, name="is_category_i_on_shelf_j")

# This is a in the proposal
# This is the total profit the store made by selling product i
profit_made_by_selling_i = model.addVars(product_id_list, vtype=GRB.CONTINUOUS, name="profit_made_by_selling_i")

# This is b in the proposal
# This is the total profit the store made from intentional demand of product i
profit_from_intentional_demand_by_selling_i = model.addVars(product_id_list, vtype=GRB.CONTINUOUS, name="profit_from_intentional_demand_by_selling_i")

# This is c in the proposal
# This is the total profit the store made from impulse demand of product i
profit_from_impulse_demand_by_selling_i = model.addVars(product_id_list, vtype=GRB.CONTINUOUS, name="profit_from_impulse_demand_by_selling_i")

# This is d in the proposal
# This is the total profit the store made from slotting fee of product i
profit_from_slotting_fees_for_i = model.addVars(product_id_list, vtype=GRB.CONTINUOUS, name="profit_from_slotting_fees_for_i")

# This determines how many different suppliers of product name i are carried by the store
number_of_suppliers_of_product_name_i_stocked = model.addVars(list(product_name_dict.keys()), vtype=GRB.INTEGER, lb=0, name="number_of_suppliers_of_product_name_i_stocked")

# This determines if product name i is stocked on shelf j
is_product_name_i_stocked_on_shelf_j = model.addVars(list(product_name_dict.keys()), shelves_set, vtype=GRB.BINARY, name="is_product_name_i_stocked_on_shelf_j")

# This determines if product name i is stocked on shelf j level k
is_product_name_i_stocked_on_shelf_j_level_k = model.addVars(list(product_name_dict.keys()), shelves_set, shelf_levels, vtype=GRB.BINARY, name="is_product_name_i_stocked_on_shelf_j_level_k")

In [16]:
# Create constraints

In [17]:
# 1
# TODO move to end
# This sets number_of_suppliers_of_product_name_i_stocked
for product_name in product_name_dict:
    expr = LinExpr()
    for product in product_name_dict[product_name]:
        expr.add(is_product_i_stocked[product.get_product_id()])
    model.addConstr(expr == number_of_suppliers_of_product_name_i_stocked[product_name], name=f"Set number_of_suppliers_of_product_name_i_stocked for product name {product_name}")

In [18]:
# Ensure atmost MAX_SUPPLIERS_PER_PRODUCT suppliers of same product are stocked
model.addConstrs((number_of_suppliers_of_product_name_i_stocked[product_name] <= MAX_SUPPLIERS_PER_PRODUCT for product_name in product_name_dict), name="Ensures atmost MAX_SUPPLIERS_PER_PRODUCT suppliers of same product are stocked");

In [19]:
# 2
# This sets is_product_name_i_stocked_on_shelf_j_level_k
for product_name in product_name_dict:
    for shelf in shelves_set:
        for level in shelf_levels:
            expr = LinExpr()
            for product in product_name_dict[product_name]:
                expr.add(is_product_i_on_shelf_j_level_k[product.get_product_id(), shelf, level])
            model.addConstr(is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, level] <= expr, name = f"Ensure that is_product_name_i_stocked_on_shelf_j_level_k[{product_name}, {shelf}, {level} is 0 if not products are stocked]")
            model.addConstr(is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, level] * LARGE_NUMBER >= expr, name = f"Ensure that is_product_name_i_stocked_on_shelf_j_level_k[{product_name}, {shelf}, {level} is 1 if products are stocked]")

In [20]:
# 3
# This sets is_product_name_i_stocked_on_shelf_j
model.addConstrs((is_product_name_i_stocked_on_shelf_j[product_name, shelf] <= is_product_name_i_stocked_on_shelf_j_level_k.sum(product_name, shelf, '*') for product_name in product_name_dict for shelf in shelves_set), name = "Ensure is_product_name_i_stocked_on_shelf_j is 0 if product not stocked");
model.addConstrs((is_product_name_i_stocked_on_shelf_j[product_name, shelf] * LARGE_NUMBER >= is_product_name_i_stocked_on_shelf_j_level_k.sum(product_name, shelf, '*') for product_name in product_name_dict for shelf in shelves_set), name = "Ensure is_product_name_i_stocked_on_shelf_j is 1 if product stocked");

In [21]:
# 4
# This ensures a product name is stocked on atmost one shelf
model.addConstrs((is_product_name_i_stocked_on_shelf_j.sum(product_name, '*') <= 1 for product_name in product_name_dict), "Ensures a product name is stocked on atmost one shelf");

In [22]:
for product_name in product_name_dict:
    for shelf in shelves_set:
        for idx in range(len(shelf_levels)):
            subarray = shelf_levels[idx:] # Make a copy from idx onwards
            if len(subarray) >= 3: # In other case we don't care
                lead_shelf = subarray[0]
                adding_shelf = subarray[1]
                subtracting_shelves = subarray[2:]
                expr = LinExpr()
                expr.add(is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, adding_shelf], NUMBER_LEVELS_PER_SHELF)
                for subtracting_shelf in subtracting_shelves:
                    expr.add(is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, subtracting_shelf], -1)
                model.addConstr(is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, lead_shelf] * NUMBER_LEVELS_PER_SHELF <= NUMBER_LEVELS_PER_SHELF + expr, name =f"Ensure continuos shelf for {product_name}, {shelf}, {lead_shelf}")
            

In [23]:
# This is (3.5.1.2) in the proposal (slightly modified as what was in there was not linear)
# Ensure that shelf i level j contains no more than NUMBER_SLOTS_PER_SHELF slots of product
for shelf in shelves_set:
    for level in shelf_levels:
        expr = LinExpr()
        for product in products:
            expr.add(displays_of_product_i_on_shelf_j_level_k[product.get_product_id(), shelf, level], product.get_slots_per_display())
        model.addConstr(expr <= NUMBER_SLOTS_PER_SHELF, name =f"Ensure shelf {shelf} level {level} contains at most NUMBER_SLOTS_PER_SHELF slots of product");

In [24]:
# This is (3.5.1.3) it ensures that each product is assigned to ATMOST one shelf and level
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') <= 1 for product in product_id_list), name="Ensure that product i is assigned to atmost one shelf");

In [25]:
# This is (3.5.1.4) it set is_product_i_stocked to 1 iff the store stocks product i
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*') == is_product_i_stocked[product] for product in product_id_list), name="Set is_product_i_stocked to 1 if product i is stocked");

In [26]:
# This is (3.5.1.5) in the proposal
# This sets is_product_i_on_shelf_level_j to 1 if product i is on shelf level j
model.addConstrs((is_product_i_on_shelf_j_level_k.sum(product, '*', level) == is_product_i_on_shelf_level_j[product, level] for product in product_id_list for level in shelf_levels), name="Set is_product_i_on_shelf_level_j to 1 if product i is on shelf level j");

In [27]:
# This is (3.5.2.3) in the proposal
# This ensures that displays_assigned_to_product_i is at most max_display
model.addConstrs((displays_assigned_to_product_i[product.get_product_id()] <= product.get_max_display() for product in products), name="Ensures that displays_assigned_to_product_i is at most max_display");

In [28]:
# This is (3.5.2.4)
# This ensures that if product i is stocked, that at least min_display quantity is stocked
model.addConstrs((displays_assigned_to_product_i[product.get_product_id()] >= is_product_i_stocked[product.get_product_id()] * product.get_min_display() for product in products), name="This ensures that if product i is stocked, that at least min_display quantity is stocked");

In [29]:
# TODO DELETE (3.5.2.5) as it is redundant due to (3.5.1.2)

In [30]:
# This is (3.5.2.6)
# This ensures constraint on displays_of_product_i_on_shelf_j_level_k
model.addConstrs((displays_of_product_i_on_shelf_j_level_k[product, shelf, level] <= NUMBER_SLOTS_PER_SHELF *  is_product_i_on_shelf_j_level_k[product, shelf, level] for product in product_id_list for shelf in shelves_set for level in shelf_levels), name="Ensure that displays_of_product_i_on_shelf_j_level_k is set correctly");

In [31]:
# This is (3.5.2.7)
# This ensures displays_assigned_to_product_i is set correctly
model.addConstrs((displays_of_product_i_on_shelf_j_level_k.sum(product, '*') == displays_assigned_to_product_i[product] for product in product_id_list), name="Ensure displays_assigned_to_product_i is set correctly");

In [32]:
# TODO DELETE, I AM REMOVING (3.5.2.8) and (3.5.2.9), these seem redundant

In [33]:
# This is (3.5.3.3)
# This ensures that atmost one category is assigned to a shelf
model.addConstrs((is_category_i_on_shelf_j.sum('*', shelf) <= 1 for shelf in shelves_set), name="This ensures at most one category is assigned to a shelf");

In [34]:

# TODO DELETE, I AM REMOVING (3.5.3.2) it seems to ensure a category is only assigned to one shelf, but we want to allow several categories per shelf

In [35]:
# This is (3.5.3.4)
# This ensures that products are only assigned to a shelf if the shelf is assigned to its category
for category in categories:
    for shelf in shelves_set:
        expr = LinExpr()
        for product in products:
            if product.get_safe_category() == category:
                for level in shelf_levels:
                    expr.add(is_product_i_on_shelf_j_level_k[product.get_product_id(), shelf, level]) 
        model.addConstr(expr <= NUMBER_LEVELS_PER_SHELF * NUMBER_SLOTS_PER_SHELF * is_category_i_on_shelf_j[category, shelf], name = f"Ensure {category} category products are on shelf {shelf} if the shelf is assigned to the category");
        # Note here that as each display of an item takes up atleast a slot, the most products a shelf could hold is NUMBER_LEVELS_PER_SHELF * NUMBER_SLOTS_PER_SHELF
        

In [36]:
# This is (3.5.4.*)
# It ensures that a product cannot be assigned to a shelf type it is not assigned to
for shelf in shelves:
    expr = LinExpr()
    for product in products:
        if product.get_safe_shelf_type() != shelf.get_shelf_type():
            for level in shelf_levels:
                expr.add(is_product_i_on_shelf_j_level_k[product.get_product_id(), shelf.get_shelf_id(), level])
    model.addConstr(expr == 0, name = f"Ensure that only products of type {shelf.get_shelf_type()} are on shelf {shelf.get_shelf_id()}");         

In [37]:
# This is (3.5.8.1)
# It ensures that the most profit we can make is the amount of items stocked plus the amount made from slotting fees
model.addConstrs((profit_made_by_selling_i[product.get_product_id()] <= product.get_units_per_display() * product.get_profit_per_unit() * displays_assigned_to_product_i[product.get_product_id()] + profit_from_slotting_fees_for_i[product.get_product_id()] for product in products), name=f"This ensures that the most profit we can make on {product.get_product_id()} is the amount of items stocked");

In [38]:
# This calculate how much is made from the intentional demand of selling product i (NOTE: This may be more than the number of products in store, hence above constraint

model.addConstrs((profit_from_intentional_demand_by_selling_i[product.get_product_id()] == PREDICT_CUSTOMERS_PER_DAY * product.get_profit_per_unit() * product.get_prob_of_intentional_purchase() for product in products), name = "Calculate profit made from intentional demand");

In [39]:
# This sets is_product_i_on_high_traffic_shelf_level_j
for product in products:
    for level in shelf_levels:
        expr = LinExpr()
        for shelf in shelves:
            if shelf.get_high_traffic():
                expr.add(is_product_i_on_shelf_j_level_k[product.get_product_id(), shelf.get_shelf_id(), level])
        model.addConstr(expr == is_product_i_on_high_traffic_shelf_level_j[product.get_product_id(), level], name = f"Determine if product {product.get_product_id()} is on level {level} of a high traffic shelf");
        

In [40]:
# This determine if a product is on a low traffic shelf, given that is_product_i_on_shelf_level_j and is_product_i_on_high_traffic_shelf_level_j are set. We can take a shortcut TODO add better explanation

model.addConstrs((is_product_i_on_low_traffic_shelf_level_j[product_name, level] + is_product_i_on_high_traffic_shelf_level_j[product_name, level] == is_product_i_on_shelf_level_j[product_name, level] for product_name in product_id_list for level in shelf_levels), name = "Set is_product_i_on_low_traffic_shelf_level_j");

In [41]:
# Determine profit_from_slotting_fees_for_i
for product in products:
    expr = LinExpr()
    for level in shelf_levels:
        expr.add(is_product_i_on_low_traffic_shelf_level_j[product.get_product_id(), level], product.get_price_to_display_on_shelf_level_i_not_on_high_traffic(level)) # Format is (variable, coeff)
        expr.add(is_product_i_on_high_traffic_shelf_level_j[product.get_product_id(), level], product.get_price_to_display_on_shelf_level_i_high_traffic(level))
        # NOTE: at most one of the above can be 1 as they are binary variable, enforced through other constraints
    model.addConstr(expr == profit_from_slotting_fees_for_i[product.get_product_id()], name=f"Calculate amount paid in slotting fees for product {product.get_product_id()}");

In [42]:
# Determine profit_from_impulse_demand_by_selling_i
for product in products:
    impulse_prob = LinExpr()
    for level in shelf_levels:
        impulse_prob.add(is_product_i_on_low_traffic_shelf_level_j[product.get_product_id(), level], product.get_prob_of_impulse_purchase() * SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I[level])
        impulse_prob.add(is_product_i_on_high_traffic_shelf_level_j[product.get_product_id(), level], product.get_prob_of_impulse_purchase() * SCALAR_MULT_FOR_FOR_BEING_ON_SHELF_LEVEL_I[level] * IMPULSE_SCALAR_MULT_FOR_HIGH_DEMAND_SHELF)
    model.addConstr(PREDICT_CUSTOMERS_PER_DAY * product.get_profit_per_unit() * impulse_prob == profit_from_impulse_demand_by_selling_i[product.get_product_id()], name = f"Calculate profit from impulse purchases of {product.get_product_id()}");
    

In [43]:
# Calculate total profit (this is a minimax)
model.addConstrs((profit_made_by_selling_i[product_name] <= profit_from_intentional_demand_by_selling_i[product_name] + profit_from_impulse_demand_by_selling_i[product_name] + profit_from_slotting_fees_for_i[product_name] for product_name in product_id_list), name = "Calculate total profit");

In [44]:
# Force store to stock essentials (this was not on proposal)
# Idea being that customers want these products and may not return to the store if they find it doesn't stock them
for product_name in essential_items:
    model.addConstr(number_of_suppliers_of_product_name_i_stocked[product_name] >= 1, name=f"Force store to stock a supplier of {product_name}")

In [45]:
# A quick objective function
model.setObjective(profit_made_by_selling_i.sum('*'), GRB.MAXIMIZE)

In [46]:
model.update()

In [47]:
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 119007 rows, 168072 columns and 870885 nonzeros
Model fingerprint: 0x5f9f401f
Variable types: 3916 continuous, 164156 integer (94456 binary)
Coefficient statistics:
  Matrix range     [9e-03, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 7e+01]
Presolve removed 58113 rows and 82428 columns (presolve time = 7s)...
Presolve removed 58113 rows and 82428 columns
Presolve time: 6.95s
Presolved: 60894 rows, 85644 columns, 392657 nonzeros
Variable types: 879 continuous, 84765 integer (51103 binary)
Found heuristic solution: objective 126.0862127
Found heuristic solution: objective 127.0366950
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...


Root simplex log...

Iteration    Objective     

In [48]:
model.write('mode9.lp')


In [49]:
# for product in products:
#     if product.get_is_essential_item():
#         print(f"product name: {product.get_product_id()}, slots per display: {product.get_slots_per_display()} min slots: {product.get_min_display()} min total {product.get_slots_per_display() * product.get_min_display()}")

In [50]:
# This will print all non-zero values
# for v in model.getVars():
#     if v.x != 0:
#         print(v.varName, "=", v.x)

In [51]:
# print_all_products()

In [52]:
def print_solution():
    for shelf in shelves:
        shelf.print()
        print("Assgined Shelf Type: ", end = "")
        for category in categories:
            if is_category_i_on_shelf_j[category, shelf.get_shelf_id()].x > 0:
                print(category)
                # Technically we can terminate loop here, but this checks that only one category was assigned to the shelf
        for level in shelf_levels:
            print(f"Products on level {level}")
            for product_name in product_id_list:
                if displays_of_product_i_on_shelf_j_level_k[product_name, shelf.get_shelf_id(), level].x > 0:
                    print()
                    print(f"    Product Name: {product_name}")
                    print(f"    Number of Displays: {displays_of_product_i_on_shelf_j_level_k[product_name, shelf.get_shelf_id(), level].x}")
                    print(f"    Slotting fee profit: {profit_from_slotting_fees_for_i[product_name].x}")
                    print(f"    Impulse Profit: {profit_from_impulse_demand_by_selling_i[product_name].x}")
                    print(f"    Intentional Demand Profit: {profit_from_intentional_demand_by_selling_i[product_name].x}")
                    print(f"    Sum of Above: {profit_from_slotting_fees_for_i[product_name].x + profit_from_impulse_demand_by_selling_i[product_name].x + profit_from_intentional_demand_by_selling_i[product_name].x}")
                    
                    print(f"    Total Profit: {profit_made_by_selling_i[product_name].x}")
    print("Products not stocked:")
    item_printed = False
    for product_name in product_id_list:
        item_printed = False
        if is_product_i_stocked[product_name].x == 0:
            print(f"    {product_name}")
            item_printed = True
    if not item_printed:
        print("    All items were stocked")
        

In [53]:
print_solution()

shelf_id: 16
shelf_type: S
high_traffic: True
Assgined Shelf Type: Bakery
Products on level 1

    Product Name: Multigrain_Bread_No_Name
    Number of Displays: 1.0
    Slotting fee profit: -0.0
    Impulse Profit: 0.6174201916092925
    Intentional Demand Profit: 1.461349565939154
    Sum of Above: 2.0787697575484465
    Total Profit: 0.9054

    Product Name: Sourdough_Bread_No_Name
    Number of Displays: 2.0
    Slotting fee profit: -0.0
    Impulse Profit: 0.6018794746307805
    Intentional Demand Profit: 1.4245668038598354
    Sum of Above: 2.026446278490616
    Total Profit: 1.8306000000000007
Products on level 2

    Product Name: Whole_Wheat_Bread_Photobug
    Number of Displays: 1.0
    Slotting fee profit: 0.24375
    Impulse Profit: 0.3117922733718092
    Intentional Demand Profit: 1.5989347352401855
    Sum of Above: 2.1544770086119946
    Total Profit: 1.8997499999999994

    Product Name: Oatmeal_Biscuit_Skajo
    Number of Displays: 1.0
    Slotting fee profit: 0.26125

In [ ]:
binding = [c for c in model.getConstrs() if abs(c.Slack) < 1e-6]



In [62]:
# for con in binding:
#     print(f"{model.getRow(con)} {con.Sense} {con.RHS}")

In [48]:
model.computeIIS()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

IIS computation: initial model status unknown, solving to determine model status
Presolve removed 38375 rows and 91093 columns
Presolve time: 0.13s

Explored 0 nodes (0 simplex iterations) in 0.38 seconds (0.56 work units)
Thread count was 1 (of 10 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0    246732         -         0    175616         -           0s
Numerical troubles encountered during IIS computation
        6         6         6

In [49]:
model.write("IIS.ilp")

model.print('X')

In [55]:
for v in model.getVars():
    if v.X != 0: # Check if the value is non-zero
        print(f"{v.VarName} = {v.X}")

is_product_i_on_shelf_j_level_k[Parmesan_Cheese_Photospace,32,1] = 1.0
is_product_i_on_shelf_j_level_k[Egg_(Quail)_Trunyx,32,2] = 1.0
is_product_i_on_shelf_j_level_k[Cheddar_Cheese_Roomm,32,5] = 1.0
is_product_i_on_shelf_j_level_k[Sunflower_Oil_Ooba,18,2] = 1.0
is_product_i_on_shelf_j_level_k[Sweet_Potato_Jaxworks,20,2] = 1.0
is_product_i_on_shelf_j_level_k[Strawberries_Skimia,19,1] = 1.0
is_product_i_on_shelf_j_level_k[Short_Grain_Rice_Tanoodle,21,2] = 1.0
is_product_i_on_shelf_j_level_k[Tomato_Fivebridge,19,5] = 1.0
is_product_i_on_shelf_j_level_k[White_Tea_No_Name,22,5] = 1.0
is_product_i_on_shelf_j_level_k[Pear_Lazz,19,2] = 1.0
is_product_i_on_shelf_j_level_k[Peach_Livepath,20,2] = 1.0
is_product_i_on_shelf_j_level_k[Greek_Yogurt_Meeveo,28,4] = 1.0
is_product_i_on_shelf_j_level_k[Carrot_Gigashots,24,5] = 1.0
is_product_i_on_shelf_j_level_k[Broccoli_Abatz,19,4] = 1.0
is_product_i_on_shelf_j_level_k[Whipped_Cream_Skippad,31,5] = 1.0
is_product_i_on_shelf_j_level_k[Cucumber_Youbridge,

In [57]:
for product_name in product_name_dict:
    for shelf in shelves_set:
        for level in shelf_levels:
            if is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, level].x > 0.5:
                print(f"is_product_name_i_stocked_on_shelf_j_level_k[{product_name}, {shelf}, {level}]: {is_product_name_i_stocked_on_shelf_j_level_k[product_name, shelf, subtracting_shelf].x}")
            

is_product_name_i_stocked_on_shelf_j_level_k[Haddock, 34, 2]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Haddock, 34, 4]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Parmesan_Cheese, 27, 3]: 1.0
is_product_name_i_stocked_on_shelf_j_level_k[Parmesan_Cheese, 27, 4]: 1.0
is_product_name_i_stocked_on_shelf_j_level_k[Parmesan_Cheese, 27, 5]: 1.0
is_product_name_i_stocked_on_shelf_j_level_k[Lime, 24, 2]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Lime, 24, 3]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Lime, 24, 4]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Arabica_Coffee, 18, 1]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Arabica_Coffee, 18, 3]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Arabica_Coffee, 18, 4]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Corn_Oil, 6, 1]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Corn_Oil, 6, 4]: -0.0
is_product_name_i_stocked_on_shelf_j_level_k[Sourdough_Bread, 12, 1]: 0.0
is_product_name_i_stoc